In [55]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import jieba
from nltk import word_tokenize

In [56]:
sklearn.__version__

'0.24.0'

In [57]:
finance_df = pd.read_csv("data/finance_data.csv")
news_df = pd.read_csv("data/news_data.csv")

In [58]:
finance_df.head()

,Unnamed: 0,ts_code,trade_date,close,open,high,low,pre_close,change,pct_chg,vol,amount
0,0,000001.SH,20201231,3473.0693,3419.7267,3474.9182,3419.7267,3414.4527,58.6166,1.7167,335673926.0,450482318.9
1,1,000001.SH,20201230,3414.4527,3375.0086,3414.4539,3374.4156,3379.0362,35.4165,1.0481,291023543.0,377542350.9
2,2,000001.SH,20201229,3379.0362,3399.2939,3407.0884,3376.0876,3397.2854,-18.2492,-0.5372,311769185.0,382102563.8
3,3,000001.SH,20201228,3397.2854,3396.3590,3412.5193,3383.6540,3396.5626,0.7228,0.0213,316181542.0,398159968.8
4,4,000001.SH,20201225,3396.5626,3351.7901,3397.0066,3348.3453,3363.1133,33.4493,0.9946,294546895.0,359094444.7


In [59]:
finance_df.shape

(243, 12)

In [60]:
news_df.head()

,Unnamed: 0,date,news,score,category
0,0,01月16,盈趣科技：利润率过于优秀,0.896216,1
1,4,01月14,2021年1月15日涨停板早知道：七大利好有望发酵,0.643451,1
2,5,01月13,2021年1月14日涨停板早知道：七大利好有望发酵,0.662349,1
3,6,01月12,2021年1月13日涨停板早知道：七大利好有望发酵,0.673256,1
4,7,01月11,2021年1月12日涨停板早知道：七大利好有望发酵,0.594757,1


In [61]:
news_df.shape

(241, 5)

In [62]:
date = news_df["date"].value_counts()
date_list = news_df["date"].values
print(len(date))
print(date)

241
09月09    1
05月06    1
10月30    1
11月17    1
06月04    1
        ..
08月09    1
11月28    1
05月31    1
09月07    1
10月21    1
Name: date, Length: 241, dtype: int64


In [63]:
news_list = news_df["news"].values
print(len(date))
print(news_list)

241
['盈趣科技：利润率过于优秀' '2021年1月15日涨停板早知道：七大利好有望发酵' '2021年1月14日涨停板早知道：七大利好有望发酵'
 '2021年1月13日涨停板早知道：七大利好有望发酵' '2021年1月12日涨停板早知道：七大利好有望发酵'
 '2021年1月11日涨停板早知道：七大利好有望发酵' '2021年1月8日涨停板早知道：七大利好有望发酵'
 '2021年1月7日涨停板早知道：七大利好有望发酵' '2021年1月6日涨停板早知道：七大利好有望发酵'
 '央行等6部委发布跨境人民币优化政策 产业迎来增量红利(附股)' '2021年1月4日涨停板早知道：七大利好有望发酵'
 '德冠新材IPO过会 禁塑令下毛利率落后同行、偿债压力在身' '2020年12月31日涨停板早知道：七大利好有望发酵'
 '2020年12月30日涨停板早知道：七大利好有望发酵' '2020年12月29日涨停板早知道：七大利好有望发酵'
 '2020年12月28日涨停板早知道：七大利好有望发酵' '我国空间站核心舱明年春季发射 航天相关个股投资价值显现(附股)'
 '2020年12月25日涨停板早知道：七大利好有望发酵' '2020年12月24日涨停板早知道：七大利好有望发酵'
 '2020年12月23日涨停板早知道：七大利好有望发酵' '3大概念爆巨大利好：国新办重磅发布白皮书 或引领市场关注这些股'
 '2020年12月21日涨停板早知道：七大利好有望发酵' '2020年12月18日涨停板早知道：七大利好有望发酵'
 '2020年12月17日涨停板早知道：七大利好有望发酵' '2020年12月16日涨停板早知道：七大利好有望发酵'
 '2020年12月15日涨停板早知道：七大利好有望发酵' '2020年12月14日涨停板早知道：七大利好有望发酵'
 '白酒、铁矿石、金融行情已轮番启动 三位一体补涨个股竟然是它' 'AI大潮将来袭：人工智能多领域应用 哪些产业链值得关注？(股)'
 '三部门深入推进互联网医疗 相关概念股有望受益' '新冠疫苗接连传利好 概念股机会又来了（附股）'
 '2022年苹果或配备潜望式长焦镜头 关注行业龙头机会' '汽车行业遇“缺芯”：国产芯片的机会来了(附股)'
 '2020年12月7日涨停板早知道：七大利好有望发酵' '“十四五”规划

In [64]:
puncts = [" ", "  ", "：","，","。","；","“","”","！","？",'!', ',' ,
          '.' ,'?' ,'-s' ,'-ly' ,'</s> ', 's', ':', ';','$', '(', ')']
def load_stopwords():
    stopwords_list = []
    filename = "data/cn_stopwords.txt"
    f = open(filename, "r", encoding='UTF-8')  
    lines = f.readlines()  
    for line in lines:  
        stopwords_list.append(line)
    return stopwords_list
    
def clean_text(x):
    x=x.strip()
    for punct in puncts:
        x=x.replace(punct,'')
    return x
 
def is_chinese(xchar):
    if xchar>=u'\u4e00' and xchar<=u'\u9fa5':
        return True
    else:
        return False
def keep_chinese_text(x):
    out_str=''
    for i in x:
        if is_chinese(i):
            out_str=out_str+i
    return out_str
 
def seg_sentence(sentence,stopwords):
    sentence_seged=jieba.cut(sentence)
    outstr=''
    for word in sentence_seged:
        if word not in stopwords:
                outstr+=word
                outstr+=" "
    return outstr

In [65]:
stopwords_list = load_stopwords()

In [66]:
news_df["news"]=news_df["news"].apply(lambda x:clean_text(x))
news_df["news"]=news_df["news"].apply(lambda x:keep_chinese_text(x))
news_df["news"]=news_df["news"].apply(lambda x:seg_sentence(x,stopwords_list))

In [67]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(news_df["news"], news_df["category"], test_size=0.25)

In [68]:
x_train

95      中德欧 领导人 明天 会晤 致力于 年内 如期完成 中欧 投资 协定 
19                年月日 涨停板 早 知道 七大 利好 有望 发酵 
220               月 日 涨停板 早 知道 七大 利好 有望 发酵 
197             亿 连接数 覆盖 扩大 这份 文件 信息量 爆棚 股 
10                年月日 涨停板 早 知道 七大 利好 有望 发酵 
                       ...                 
169               月 日 涨停板 早 知道 七大 利好 有望 发酵 
157    数据安全 立法 再进一步 草案 初次 提请 审议 这些 龙头 或 受益 
179               月 日 涨停板 早 知道 七大 利好 有望 发酵 
224         国信 证券 持续 看好 华为 产业链 维持 行业 超配 评级 
1                 年月日 涨停板 早 知道 七大 利好 有望 发酵 
Name: news, Length: 180, dtype: object

In [69]:
y_train

95     1
19     1
220    1
197    0
10     1
      ..
169    1
157    1
179    1
224    1
1      1
Name: category, Length: 180, dtype: int64

In [70]:
train_d = {'data':x_train,
       'label':y_train}
test_d = {'data':x_test,
       'label':y_test}

train_df = pd.DataFrame(train_d)
test_df = pd.DataFrame(test_d)

In [71]:
train_df.head()

,data,label
95,中德欧 领导人 明天 会晤 致力于 年内 如期完成 中欧 投资 协定,1
19,年月日 涨停板 早 知道 七大 利好 有望 发酵,1
220,月 日 涨停板 早 知道 七大 利好 有望 发酵,1
197,亿 连接数 覆盖 扩大 这份 文件 信息量 爆棚 股,0
10,年月日 涨停板 早 知道 七大 利好 有望 发酵,1


In [72]:
test_df.head()

,data,label
148,月 日 涨停板 早 知道 七大 利好 有望 发酵,1
36,年月日 涨停板 早 知道 七大 利好 有望 发酵,1
65,年月日 涨停板 早 知道 七大 利好 有望 发酵,1
11,德冠新材 过会禁 塑令 下 毛利率 落后 同行 偿债 压力 在 身,0
189,零食 股大 三只 松鼠 良品 铺子 百草 味 谁 领风骚,1


In [73]:
train_df["data"].value_counts()

年月日 涨停板 早 知道 七大 利好 有望 发酵                       70
月 日 涨停板 早 知道 七大 利好 有望 发酵                       58
亿 连接数 覆盖 扩大 这份 文件 信息量 爆棚 股                      1
华为 鸿蒙 车载 系统 揭开 面纱 未来 操作系统 将成 汽车 发展 灵魂           1
新冠 疫苗 接连 传 利好 概念股 机会 又 来 了 附股                   1
广州 开展 智能网 联 汽车 道路 测试 产业链 相关 公司 有望 受益            1
产业 人士 预警 模拟 芯片 产能 告急 本土 公司 机遇 显现                1
国信 证券 汽车 汽配 进入 高性价比 配置 阶段 精选 主线                 1
资金 本周 回流 白酒 股遭 爆买 这些 板块 获 大幅 加仓 附股              1
视频 多地 推出 鼓励 购车 政策 这些 相关 概念股 值得 关注               1
成功 挤掉 苹果 小米 手机 全球 销量 挺进 前 三 背后 有 这些 供应商         1
复工 不到 一个月 电影院线 再迎 利好 龙头股 迎 估值 修复 机会             1
华为 发布 亚米级 车道 导航 高精度 定位 商用 落地 再进一步 名单            1
汽车行业 遇缺 芯 国产 芯片 的 机会 来 了 附股                     1
水泥 涨价 模式 开启 相关 概念股 有望 受益 股                      1
顺 周期 概念股 全面 爆发 还有 哪些 板块 可 挖掘                    1
中信证券 维持 歌尔 股份 股票 买入 评级                          1
年 苹果 或 配备 潜望 式 长焦 镜头 关注 行业龙头 机会                 1
一季度 险资 大笔 加仓 累计 举牌 次 青睐 银行 材料                   1
头部 影片 定档 助推 电影 市场 复苏 两 主线 掘金 相关 板块 机会           1


In [74]:
def createVocabList(dataSet):
    vocabSet = set([])
    for document in dataSet:
        vocabSet = vocabSet | set(document)
    return list(vocabSet)

def bagOfWords2Vec(vocabList, inputSet):
    returnVec = [0] * len(vocabList)
    for word in inputSet:
        if word in vocabList:
            returnVec[vocabList.index(word)] += 1
        else:
            print("the word: %s is not in my vocabulary!" % word)
        return returnVec   

In [76]:
all_cut_seg = []
all_cut_words = [] # unique
all_cut_raw_words = [] # all

for text in train_df["data"].values:   
    word_list = word_tokenize(text)
    all_cut_seg.append(word_list)
    for word in word_list:
        if (word.lower() not in stopwords_list):
            all_cut_raw_words.append(word.lower())
            if word not in all_cut_words:
                all_cut_words.append(word.lower())

In [80]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
vectorizer.fit(train_df["data"].values)
X_train = vectorizer.transform(train_df["data"].values)
X_test  = vectorizer.transform(test_df["data"].values)
label_train = train_df["label"].values
label_test = test_df["label"].values
print(X_train.shape)
print(X_train[0].toarray())

(180, 358)
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]]


In [81]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.metrics import confusion_matrix

classifier1 = MultinomialNB()
classifier1.fit(X_train, label_train)
score11 = classifier1.score(X_test, label_test)
score12 = classifier1.score(X_train, label_train)
print("Accuracy of Naive Bayes model:", score12)

Accuracy of Naive Bayes model: 0.8555555555555555


In [82]:
result_svm = classifier1.predict(X_test)
print(confusion_matrix(label_test, result_svm))

[[ 2  2]
 [ 2 55]]


In [83]:
classifier2 = LinearSVC()
classifier2.fit(X_train, label_train)
score21 = classifier2.score(X_test, label_test)
score22 = classifier2.score(X_train, label_train)
print("Accuracy of SVM model:", score22)

Accuracy of SVM model: 0.9888888888888889


In [84]:
result_svm = classifier2.predict(X_test)
print(confusion_matrix(label_test, result_svm))

[[ 1  3]
 [ 2 55]]
